In [4]:
import requests

url = "https://live.trading212.com/api/v0/equity/portfolio"

headers = {"Authorization": ""}

response = requests.get(url, headers=headers)

data = response.json()
print(data)

[{'ticker': 'LGENl_EQ', 'quantity': 50.0, 'averagePrice': 252.8, 'currentPrice': 224.5, 'ppl': -14.15, 'fxPpl': None, 'initialFillDate': '2024-02-01T10:00:31.000+02:00', 'frontend': 'IOS', 'maxBuy': 1037535.0, 'maxSell': 1037585.0, 'pieQuantity': 0.0}, {'ticker': 'NESNs_EQ', 'quantity': 0.773999, 'averagePrice': 92.33862059, 'currentPrice': 84.32, 'ppl': -5.14, 'fxPpl': 0.37, 'initialFillDate': '2024-02-01T10:01:38.000+02:00', 'frontend': 'AUTOINVEST', 'maxBuy': 197337.226001, 'maxSell': 197338.0, 'pieQuantity': 0.773999}, {'ticker': 'SKX_US_EQ', 'quantity': 1.0, 'averagePrice': 63.24, 'currentPrice': 65.7, 'ppl': 1.33, 'fxPpl': -0.55, 'initialFillDate': '2024-07-22T17:45:50.000+03:00', 'frontend': 'WEB', 'maxBuy': 73993.0, 'maxSell': 73994.0, 'pieQuantity': 0.0}, {'ticker': 'AAPL_US_EQ', 'quantity': 1.4198586, 'averagePrice': 187.27216921, 'currentPrice': 231.22, 'ppl': 43.55, 'fxPpl': -4.23, 'initialFillDate': '2024-01-25T17:46:14.000+02:00', 'frontend': 'AUTOINVEST', 'maxBuy': 31272